In [1]:
import tools
import yfinance as yf
import pandas as pd

In [2]:
con = tools.connect_postgres()

In [3]:
t = yf.Ticker("AAPL")
tools.insert_symbol(con, t)

Eintrag existiert schon


In [4]:
t.quarterly_earnings

,Revenue,Earnings
Quarter,,
3Q2021,83360000000,20551000000
4Q2021,123945000000,34630000000
1Q2022,97278000000,25010000000
2Q2022,82959000000,19442000000


In [22]:
# Tabelle aus Ticker holen
df = t.quarterly_earnings
# Umformen, damit passend zu SQL-Tabelle
symbol = t.info["symbol"]
df["symbol"] = symbol
df["quarter"] = df.index
df["quarter"] = df["quarter"].str[2:] + "Q" + df["quarter"].str[0]
df = df.rename(columns={"Revenue":"revenue", "Earnings":"earnings"})
# in Datenbank hochladen
df.to_sql("earnings_quarter", con, 
    schema="finance", if_exists="append", index=False)

4

In [14]:
#df["quarter"].str[2:]
#df["quarter"].str[0]
df["quarter"].str[2:] + "Q" + df["quarter"].str[0]


Quarter
3Q2021    2021Q3
4Q2021    2021Q4
1Q2022    2022Q1
2Q2022    2022Q2
Name: quarter, dtype: object

In [25]:
pd.read_sql("select * from finance.earnings_quarter",con)

,symbol,quarter,revenue,earnings
0,AAPL,2021Q3,83360000000,20551000000
1,AAPL,2021Q4,123945000000,34630000000
2,AAPL,2022Q1,97278000000,25010000000
3,AAPL,2022Q2,82959000000,19442000000
